In [1]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "testpassword"

driver = GraphDatabase.driver(uri, auth=(username, password))

with driver.session() as session:
    result = session.run("MATCH (n) RETURN n")
    for record in result:
        print(record)
driver.close()

# Menu

1) <a href="#insert">Insert ad-hoc data</a>
2) <a href="#csv">Load a CSV of data</a>
3) <a href="#query">Query database</a>

<a name="insert" />

# Insert ad-hoc data

In [4]:
from neo4j import GraphDatabase

# Connect to the Neo4j instance
uri = "bolt://localhost:7687"
username = "neo4j"
password = "testpassword"

driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to add data
def add_data(session):
    session.run(
        """
        CREATE (a:Person {name: 'Alice', age: 30})
        CREATE (b:Person {name: 'Bob', age: 25})
        CREATE (c:Person {name: 'Charlie', age: 35})
        CREATE (a)-[:FRIENDS_WITH]->(b)
        CREATE (b)-[:FRIENDS_WITH]->(c)
        """
    )

# Function to query data
def query_data(session):
    result = session.run(
        """
        MATCH (p:Person)-[:FRIENDS_WITH]->(friend)
        RETURN p.name AS person, friend.name AS friend
        """
    )
    for record in result:
        print(f"{record['person']} is friends with {record['friend']}")

# Run the add and query functions in a session
with driver.session() as session:
    add_data(session)
    query_data(session)

# Close the driver connection
driver.close()


Alice is friends with Bob
Bob is friends with Charlie


<a name="csv" />

# Load a CSV of data

`nodes.csv`
```
id,name,age
1,Alice,30
2,Bob,25
3,Charlie,35
```

`relationships.csv`
```
start_id,end_id,relationship
1,2,FRIENDS_WITH
2,3,FRIENDS_WITH
```

In [ ]:
LOAD CSV WITH HEADERS FROM 'file:///nodes.csv' AS row
CREATE (:Person {id: toInteger(row.id), name: row.name, age: toInteger(row.age)});

In [ ]:
LOAD CSV WITH HEADERS FROM 'file:///relationships.csv' AS row
MATCH (a:Person {id: toInteger(row.start_id)}), (b:Person {id: toInteger(row.end_id)})
CREATE (a)-[:FRIENDS_WITH]->(b);

<a name="query" />

# Query database